In [1]:
import requests
import json
import pandas as pd
import yaml 
import time


In [2]:
with open('psw/token.yml', 'r') as f:
    token_file = yaml.safe_load(f)

TOKEN = token_file['TOKEN API FOOTBALL']



url = 'https://v3.football.api-sports.io/fixtures'
url_stat = "https://v3.football.api-sports.io/teams/statistics"
headers = { 'x-rapidapi-key': TOKEN }


In [3]:
years = [2021,2022,2023]

matches_data = []
stat_data = []
formations_data = []
requests_count = 0

for year in years:
    params = {'season':year
              ,'league': 135}
    response = requests.get(url, headers=headers, params=params)
    requests_count += 1

    if requests_count % 10 == 0:
        time.sleep(60)

    if response.status_code != 200:
        print(f"Errore nella richiesta: {response.status_code}")

    data = response.json()
    matches = data.get('response',[])
    
    diz_id = {}
    for match in matches:
        date = match["fixture"]["date"].split("T")[0]
        giornata = match["league"]["round"]
        home_team = match["teams"]["home"]["name"]
        home_winner = match["teams"]["home"]["winner"]
        away_team = match["teams"]["away"]["name"]
        away_winner = match["teams"]["away"]["winner"]
        home_goals = match["goals"]["home"]
        away_goals = match["goals"]["away"]
        home_goals_halftime = match["score"]["halftime"]["home"]
        away_goals_halftime = match["score"]["halftime"]["away"]
        penalty_home = match["score"]["penalty"]["home"]
        penalty_away = match["score"]["penalty"]["away"]
        id_home = match["teams"]["home"]["id"]


        if id_home not in diz_id:
            diz_id[id_home] = home_team

        matches_data.append({
            "season": year,
            "date": date,
            "matchweek": giornata,
            "home_team": home_team,
            "home_winner": home_winner,
            "away_team":away_team,
            "away_winner":away_winner,
            "home_goals": home_goals,
            "away_goals": away_goals,
            "home_goals_halftime": home_goals_halftime,
            "away_goals_halftime": away_goals_halftime})

In [4]:
df_matches = pd.DataFrame(matches_data)
df_matches.to_csv('dataset/matches_history.csv', index= False)